In [1]:
import numpy as np
import pandas as pd

In [2]:
def generate_2mmpp_trace(T_max, lambda0, lambda1, q01, q10, seed=None):
    """
    Generate a 2-state MMPP traffic trace.

    Parameters:
    - T_max (float): total simulation time (in seconds)
    - lambda0 (float): Poisson rate in state 0
    - lambda1 (float): Poisson rate in state 1
    - q01 (float): transition rate from state 0 to state 1
    - q10 (float): transition rate from state 1 to state 0
    - seed (int or None): random seed

    Returns:
    - arrivals (list of float): timestamps of Poisson arrivals
    - state_times (list of tuples): (start_time, end_time, state, rate) for each state period
    """
    if seed is not None:
        np.random.seed(seed)

    arrivals = []
    state_times = []
    current_time = 0.0
    state = np.random.choice([0, 1])  # randomly start in state 0 or 1

    while current_time < T_max:
        # Determine time until next state change
        if state == 0:
            holding_time = np.random.exponential(1.0 / q01)
            rate = lambda0
        else:
            holding_time = np.random.exponential(1.0 / q10)
            rate = lambda1

        end_time = min(current_time + holding_time, T_max)

        # Generate Poisson arrivals in this state
        duration = end_time - current_time
        num_arrivals = np.random.poisson(rate * duration)
        arrivals_in_state = np.sort(np.random.uniform(current_time, end_time, num_arrivals))
        arrivals.extend(arrivals_in_state)
        state_times.append((current_time, end_time, state, rate))

        # Update time and switch state
        current_time = end_time
        state = 1 - state  # switch state

    return np.array(arrivals), state_times

def find_arrival_rates_from_data(arrivals, state_segments):
    arrival_rates_from_data = []
    curr_idx = 0
    for a in arrivals:
        low= state_segments[curr_idx][0]
        high = state_segments[curr_idx][1]
        rate= state_segments[curr_idx][3]
        if(low<=a<high):
            arrival_rates_from_data.append(rate)
        else:
            curr_idx+=1
            rate = state_segments[curr_idx][3]
            arrival_rates_from_data.append(rate)
    return arrival_rates_from_data

def trace(info_dict):
    types = len(info_dict)
    total_df = pd.DataFrame()

    for i in range(types):
        T_max = info_dict[i]['T_max']
        lambda0 = info_dict[i]['lambda0']
        lambda1 = info_dict[i]['lambda1']
        q01 = info_dict[i]['q01']
        q10 = info_dict[i]['q10']
        num_prefill_tokens = info_dict[i]['num_prefill_tokens']
        num_decode_tokens = info_dict[i]['num_decode_tokens']
        arrivals, state_segments = generate_2mmpp_trace(T_max, lambda0, lambda1, q01, q10, seed=None)
        arrival_rates = find_arrival_rates_from_data(arrivals=arrivals, state_segments=state_segments)

        csv_arrivals= pd.DataFrame(arrivals)
        csv_arrivals['num_prefill_tokens'] = num_prefill_tokens
        csv_arrivals['num_decode_tokens'] = num_decode_tokens
        csv_arrivals['prompt_type'] = f'type{i+1}'
        csv_arrivals['arrival_rate_update'] = arrival_rates
        csv_arrivals = csv_arrivals.rename(columns={0: 'arrived_at'})

        total_df = pd.concat([total_df, csv_arrivals])

    total_df = total_df.sort_values(by='arrived_at')
    return total_df

In [3]:
info_dict_input = [
    {'T_max': 10, 'lambda0': 1000, 'lambda1': 500, 'q01': 0.5, 'q10': 0.3, 'num_prefill_tokens': 10, 'num_decode_tokens': 10},
    {'T_max': 10, 'lambda0': 1000, 'lambda1': 200, 'q01': 0.5, 'q10': 0.3, 'num_prefill_tokens': 10, 'num_decode_tokens': 20},
]

trace_mmpp2= trace(info_dict=info_dict_input)

In [4]:
info_dict_input = [
    {'T_max': 9, 'lambda0': 5000, 'lambda1': 4000, 'q01': 0.5, 'q10': 0.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 100},
    {'T_max': 9, 'lambda0': 4000, 'lambda1': 3000, 'q01': 0.5, 'q10': 0.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 200},
    {'T_max': 9, 'lambda0': 2000, 'lambda1': 1000, 'q01': 0.5, 'q10': 0.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 300},
]

trace_mmpp2= trace(info_dict=info_dict_input)

In [24]:
info_dict_input = [
    {'T_max': 10, 'lambda0': 1000, 'lambda1': 500, 'q01': 1.0, 'q10': 0.8, 'num_prefill_tokens': 10, 'num_decode_tokens': 10},
    {'T_max': 10, 'lambda0': 1000, 'lambda1': 200, 'q01': 1.0, 'q10': 0.8, 'num_prefill_tokens': 10, 'num_decode_tokens': 20},
]

trace_mmpp2= trace(info_dict=info_dict_input)

In [6]:
info_dict_input = [
    {'T_max': 1.5, 'lambda0': 5000, 'lambda1': 4000, 'q01': 1.5, 'q10': 1.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 100},
    {'T_max': 1.5, 'lambda0': 4000, 'lambda1': 3000, 'q01': 1.5, 'q10': 1.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 200},
    {'T_max': 1.5, 'lambda0': 2000, 'lambda1': 1000, 'q01': 1.5, 'q10': 1.3, 'num_prefill_tokens': 20, 'num_decode_tokens': 300},
]

trace_mmpp2= trace(info_dict=info_dict_input)

In [7]:
trace_mmpp2.to_csv('high_demand_high_shift_wait.csv')

In [8]:
for i in range(len(info_dict_input)):
    print(np.mean(trace_mmpp2[trace_mmpp2['prompt_type'] == f'type{i+1}']['arrival_rate_update']))

4877.736472532011
3381.254964257347
1176.1876127480457


In [9]:
print(trace_mmpp2)

      arrived_at  num_prefill_tokens  num_decode_tokens prompt_type  \
0       0.000025                  20                100       type1   
1       0.000065                  20                100       type1   
2       0.000227                  20                100       type1   
0       0.000533                  20                200       type2   
3       0.000625                  20                100       type1   
...          ...                 ...                ...         ...   
5033    1.499475                  20                200       type2   
5034    1.499492                  20                200       type2   
7261    1.499654                  20                100       type1   
5035    1.499894                  20                200       type2   
7262    1.499982                  20                100       type1   

      arrival_rate_update  
0                    5000  
1                    5000  
2                    5000  
0                    4000  
3      

In [68]:
info_dict_input = [
    {'T_max': 10, 'lambda0': 1000, 'lambda1': 100, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 25},
    {'T_max': 10, 'lambda0': 900, 'lambda1': 90, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 50},
    {'T_max': 10, 'lambda0': 800, 'lambda1': 80, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 100},
    {'T_max': 10, 'lambda0': 700, 'lambda1': 70, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 200},
    {'T_max': 10, 'lambda0': 600, 'lambda1': 60, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 400},
    {'T_max': 10, 'lambda0': 500, 'lambda1': 50, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 500},
    {'T_max': 10, 'lambda0': 400, 'lambda1': 40, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 600},
    {'T_max': 10, 'lambda0': 300, 'lambda1': 30, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 700},
    {'T_max': 10, 'lambda0': 200, 'lambda1': 20, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 800},
    {'T_max': 10, 'lambda0': 100, 'lambda1': 10, 'q01': 5, 'q10': 5, 'num_prefill_tokens': 100, 'num_decode_tokens': 900},
]

trace_mmpp2= trace(info_dict=info_dict_input)

In [69]:
trace_mmpp2.to_csv('high_rate_ten_type_mmpp2.csv')

In [70]:
print(trace_mmpp2.head())

   arrived_at  num_prefill_tokens  num_decode_tokens prompt_type  \
0    0.000164                 100                600       type7   
0    0.000241                 100                 50       type2   
1    0.000305                 100                 50       type2   
0    0.000390                 100                 25       type1   
0    0.000788                 100                500       type6   

   arrival_rate_update  
0                  400  
0                  900  
1                  900  
0                 1000  
0                  500  


In [71]:
for i in range(len(info_dict_input)):
    print(np.mean(trace_mmpp2[trace_mmpp2['prompt_type'] == f'type{i+1}']['arrival_rate_update']))

901.2411347517731
823.0772366007744
758.9317054559328
647.843137254902
546.1538461538462
464.6900752418488
382.93838862559244
263.5459940652819
180.74342701722574
91.08571428571429
